In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load the model during application startup
model = joblib.load('model.pkl')

@app.route("/")
def hello():
    return "Hello from the last session in our course"

@app.route("/predict_heart", methods=["POST"])
def predict_heart():
    try:
        if request.method == 'POST':
            # Check Content-Type
            content_type = request.headers.get('Content-Type')
            if not content_type or 'application/json' not in content_type:
                return jsonify({"error": "Unsupported Media Type: Content-Type should be 'application/json'"}), 415  # 415 is the HTTP status code for Unsupported Media Type
            
            data = request.get_json()  # Get JSON data
            if not data:
                return jsonify({"error": "No JSON data provided"}), 400  # 400 is the HTTP status code for Bad Request

            # Feature extraction
            features = ["bmi", "Smoking", "Stroke", "PhysicalHealth", "Sex", "AgeCategory", "Diabetic", "GenHealth", "SleepTime"]
            
            # Ensure data types are correct
            input_data = [
                float(data.get("bmi", 0)),            # Assuming "bmi" is a float
                int(data.get("Smoking", 0)),           # Assuming "Smoking" is an integer
                int(data.get("Stroke", 0)),            # Assuming "Stroke" is an integer
                float(data.get("PhysicalHealth", 0)),  # Assuming "PhysicalHealth" is a float
                int(data.get("Sex", 0)),               # Assuming "Sex" is an integer
                int(data.get("AgeCategory", 0)),       # Assuming "AgeCategory" is an integer
                int(data.get("Diabetic", 0)),          # Assuming "Diabetic" is an integer
                int(data.get("GenHealth", 0)),         # Assuming "GenHealth" is an integer
                float(data.get("SleepTime", 0))        # Assuming "SleepTime" is a float
            ]

            # Model prediction
            prediction = model.predict([input_data])[0]

            # Interpret the prediction
            result_string = "Positive for Heart Disease" if prediction == 1 else "Negative for Heart Disease" 

            return jsonify({"result": result_string})

    except Exception as e:
        return jsonify({"error": str(e)}), 500  # 500 is the HTTP status code for Internal Server Error

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

